# Geospatial_04_exercise

[Starbucks Reserve Roastery](스타벅스 리저브 로스터리)의 다음 매장을 찾고 있는 스타벅스 빅데이터 분석가([그것이 진짜 직업이다!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)(https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5).   
이 로스터리들은 일반적인 스타벅스 매장보다 훨씬 더 크고 고급 라운지 구역과 함께 다양한 음식과 와인 옵션을 포함한 몇 가지 추가적인 특징들을 가지고 있다.  캘리포니아 주에 있는 여러 카운티의 인구 통계를 조사하여 잠재적으로 적합한 위치를 결정합니다.

<center>
<img src="https://i.imgur.com/BIyE6kR.png" width="450"><br/><br/>
</center>

시작하기 전에 아래 코드 셀을 실행하여 모든 것을 설정하십시오.

In [1]:
import math
import pandas as pd
import geopandas as gpd
#from geopy.geocoders import Nominatim            # What you'd normally run
from learntools.geospatial.tools import Nominatim # Just for this exercise

import folium 
from folium import Marker
from folium.plugins import MarkerCluster

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex4 import *

이전 연습의 embed_map() 함수를 사용하여 지도를 시각화합니다.

In [2]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

### 1) 누락된 위치를 지오코딩합니다.

다음 코드 셀을 실행하여 캘리포니아 주에 있는 스타벅스 매장을 포함하는 데이터 프레임 "스타벅스"를 만드십시오.

In [3]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("data/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


대부분의 상점들은 위도, 경도 등의 위치를 알고 있다.  하지만, 버클리 시의 모든 장소가 사라졌습니다.

In [4]:
# How many rows in each column have missing values?
print(starbucks.isnull().sum())

# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]
rows_with_missing

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64


,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


아래 코드 셀을 사용하여 이 값을 Nominatim 지오코더로 채우십시오.

튜토리얼에서 우리는 지오코드 값에 "Nominatim()"("geopy.geocoders"의)을 사용했으며, 이것이 이 과정 이외의 프로젝트에서 사용할 수 있는 것이다.  

이 연습에서는 learntools.geospacial과는 약간 다른 함수인 "Nominatim()"을 사용합니다.툴').  이 기능은 노트북 상단에 가져온 것으로 GeoPandas의 기능과 동일하게 작동합니다.

다시 말해서, 다음과 같은 경우: 
- 노트북 맨 위에 있는 가져오기 문을 변경하지 않습니다. 
- 당신은 아래의 코드 셀에서 지오코딩 함수를 "geocode`"라고 부른다. 

코드가 의도한 대로 작동합니다!

In [5]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

# Your code here
def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Longitude': point.longitude, 'Latitude': point.latitude})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)
# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 2) 버클리의 위치를 확인합니다.

당신이 방금 찾은 장소들을 살펴봅시다.  버클리의 (위도, 경도) 위치를 OpenStreetMap 스타일로 시각화합니다.

In [6]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Your code here: Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)
# Get credit for your work after you have created a map
q_2.a.check()

# Show the map
m_2

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Thank you for creating a map!</span>

버클리의 5개 위치만 고려했을 때, (위도, 경도) 위치 중 몇 개가 (올바른 도시에 위치) 잠재적으로 맞는 것 같습니까?

In [7]:
# View the solution (Run this code cell to receive credit!)
q_2.b.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> All five locations appear to be correct!

### 3) 데이터 통합

아래 코드를 실행하여 캘리포니아 주의 각 카운티에 대한 이름, 면적(제곱킬로미터 단위) 및 고유 ID("GEOID" 열에 있음)가 포함된 GeoDataFrame "CA_counties"를 로드합니다.  지오메트리 열에는 군 경계를 가진 다각형이 포함됩니다.

In [8]:
CA_counties = gpd.read_file("data/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.crs = {'init': 'epsg:4326'}
CA_counties.head()

c:\Users\User\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


그런 다음 세 가지 데이터 프레임을 생성합니다.
- `CA_pop`은 각 군의 인구 추계를 담고 있다.
- `CA_high_earners`에는 연간 최소 15만 달러의 소득이 있는 가구 수가 포함되어 있다.
- `CA_median_age`에는 각 카운티의 중위연령이 포함됩니다.

In [9]:
CA_pop = pd.read_csv("data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("data/CA_county_median_age.csv", index_col="GEOID")

다음 코드 셀을 사용하여 'CA_pop', 'CA_high_earners', 'CA_median_age'와 'CA_counties' GeoDataFrame을 결합합니다.

생성된 GeoDataFrame의 이름을 'CA_stats'로 지정하고 8개의 열('GEOID', '이름', 'area_sqkm', 'geometry', '인구', 'high_earners', 'median_age')이 있는지 확인합니다.

In [11]:
# Your code here
CA_stats = CA_counties.set_index("GEOID", inplace=False).join([CA_high_earners, CA_median_age, CA_pop]).reset_index()

# Check your answer
q_3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

이제 모든 데이터를 한 곳에 모아 놓았으므로 열의 조합을 사용하는 통계량을 계산하는 것이 훨씬 쉬워졌습니다.  다음 코드 셀을 실행하여 모집단 밀도가 있는 "밀도" 열을 만듭니다.

In [12]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]

### 4) 어느 나라가 유망해 보이나요?

모든 정보를 단일 GeoDataFrame으로 축소하면 특정 기준에 맞는 카운티를 훨씬 쉽게 선택할 수 있습니다.

다음 코드 셀을 사용하여 "CA_stats" GeoDataFrame의 행 부분 집합(및 모든 열)을 포함하는 GeoDataFrame 'sel_counties'를 만듭니다.  특히 다음과 같은 카운티를 선택해야 합니다.
- 적어도 100,000가구가 매년 150,000원을 번다.
- 중위연령은 38.5세 미만이다.
- 인구 밀도는 최소 285명(평방킬로미터당)이다.

또한, 선택된 카운티는 다음 기준 중 적어도 하나를 충족해야 합니다.
- 적어도 50만 가구가 매년 150,000원을 번다.
- 중위연령이 35.5세 미만 또는
- 거주자의 밀도는 최소한 1400 (평방 킬로미터 당)이다.

In [13]:
# Your code here
sel_counties = CA_stats[((CA_stats.high_earners > 100000) & \
                         (CA_stats.median_age < 38.5) & \
                         (CA_stats.density > 285) & \
                        ((CA_stats.median_age < 35.5) | \
                         (CA_stats.density > 1400) | \
                         (CA_stats.high_earners > 500000)))]
# Check your answer
q_4.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 5) 몇 개의 매장을 확인하셨나요?

다음 Starbucks Reserve Roastery 위치를 찾을 때 선택한 카운티 내의 모든 매장을 고려하려고 합니다.  그렇다면, 선택된 카운티 내에 몇 개의 상점이 있을까요?

이 질문에 대답할 준비를 하려면 다음 코드 셀을 실행하여 모든 스타벅스 위치가 포함된 GeoDataFrame 'starbucks_gdf'를 생성하십시오.

In [14]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

c:\Users\User\anaconda3\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


그래서, 당신이 선택한 카운티에는 몇 개의 상점이 있나요?

In [15]:
# Fill in your answer
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)

# Check your answer
q_5.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

### 6) 상점 위치를 시각화합니다.

이전 질문에서 식별한 스토어의 위치를 표시하는 맵을 만듭니다.

In [16]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Your code here: show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))
        
m_6.add_child(mc)
# Get credit for your work after you have created a map
q_6.check()

# Show the map
m_6

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Thank you for creating a map!</span>

# Keep going

Learn about how **[proximity analysis](#$NEXT_NOTEBOOK_URL$)** can help you to understand the relationships between points on a map.